<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/Clasificacion_Clase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# JFK Delay prediction

Importe librerias

In [126]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import libraries for plotting
import matplotlib.pyplot as plt
import seaborn as sns
# from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
pd.set_option('display.max_rows', 500)
pd.options.display.float_format = '{:,.2f}'.format


También cargaremos todas las funciones de preprocesamientoo de scikit-learn, contenidas en el **scikit preprocessing**.


In [127]:
from sklearn import preprocessing

Y aprovecharemos para comprobar la **versión instalada de scikit-learn**.

In [128]:
from sklearn import __version__ as sklearn_version
print('La versión instalada de scikit-learn es {}.'.format(sklearn_version))

La versión instalada de scikit-learn es 1.6.1.


Antes de empezar a repasar la Machine Learning Checklist, importatemos el DataSet de trabajo (**JFKFlights**). Este Dataset se encuentra publicado en la página oficial del Departamento de transporte de Estados Unidos , aunque para facilitar el trabajo lo hemos trasladado a un link de descarga en internet (repositorio público en GitHub).

In [129]:
url1 = 'https://bit.ly/3bf6QNP'
url2 = 'https://bit.ly/3kLR2p3'
url_weather = 'https://raw.githubusercontent.com/Sitg35/Binary-Classifier-JFK/refs/heads/main/NYC_Weather_2016_2022.csv'

In [130]:
df1= pd.read_csv(url1, index_col=0)
#lee la url el csv, con el df1= hacemos q se guarde

In [131]:
df2= pd.read_csv(url2, index_col=0)

In [132]:
df3= pd.read_csv(url_weather, index_col=0)

Unir dos csv, se tuvieron q partir por tamaño

In [133]:
df= pd.concat([df1, df2], ignore_index=True)

In [134]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
OP_CARRIER_AIRLINE_ID,"237,644.00","20,152.00",297.88,"19,690.00","19,790.00","20,363.00","20,409.00","21,171.00"
CRS_DEP_TIME,"250,152.00","1,334.61",529.10,3.00,915.00,"1,258.00","1,744.00","2,359.00"
TAXI_OUT,"232,641.00",18.19,9.61,1.00,12.00,16.00,21.00,166.00
CRS_ARR_TIME,"250,152.00","1,419.13",580.99,1.00,944.00,"1,455.00","1,914.00","2,400.00"
ARR_DEL15,"250,152.00",0.21,0.41,0.00,0.00,0.00,0.00,1.00
CRS_ELAPSED_TIME,"250,152.00",195.65,97.37,47.00,109.00,167.00,306.00,595.00
DISTANCE,"250,152.00","1,258.84",898.12,94.00,427.00,"1,028.00","2,248.00","4,983.00"


In [135]:
df.reset_index(drop = True, inplace = True)

In [136]:
df_weather = pd.read_csv(url_weather) #granularidad fecha y hora
print(df_weather)

                   time  temperature_2m (°C)  precipitation (mm)  rain (mm)  \
0      2016-01-01T00:00                 7.60                0.00       0.00   
1      2016-01-01T01:00                 7.50                0.00       0.00   
2      2016-01-01T02:00                 7.10                0.00       0.00   
3      2016-01-01T03:00                 6.60                0.00       0.00   
4      2016-01-01T04:00                 6.30                0.00       0.00   
...                 ...                  ...                 ...        ...   
59755  2022-10-25T19:00                  NaN                 NaN        NaN   
59756  2022-10-25T20:00                  NaN                 NaN        NaN   
59757  2022-10-25T21:00                  NaN                 NaN        NaN   
59758  2022-10-25T22:00                  NaN                 NaN        NaN   
59759  2022-10-25T23:00                  NaN                 NaN        NaN   

       cloudcover (%)  cloudcover_low (%)  cloudcov

In [137]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59760 entries, 0 to 59759
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   time                   59760 non-null  object 
 1   temperature_2m (°C)    59592 non-null  float64
 2   precipitation (mm)     59592 non-null  float64
 3   rain (mm)              59592 non-null  float64
 4   cloudcover (%)         59592 non-null  float64
 5   cloudcover_low (%)     59592 non-null  float64
 6   cloudcover_mid (%)     59592 non-null  float64
 7   cloudcover_high (%)    59592 non-null  float64
 8   windspeed_10m (km/h)   59592 non-null  float64
 9   winddirection_10m (°)  59587 non-null  float64
dtypes: float64(9), object(1)
memory usage: 4.6+ MB


Transforma el índice actual de df_weather (que contiene la fecha y hora) en una columna llamada 'time'.
Esto crea un nuevo índice numérico por defecto

 Convierte los valores de la columna 'time' a tipo datetime, permitiendo operaciones de series temporales.
 NOTA: Para que la conversión se guarde en el DataFrame, debes asignar el resultado a df_weather['time'].

In [138]:
df_weather['time'] = pd.to_datetime(df_weather['time'])

DATA JOIN

In [139]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])
#LA PONEMOS COMO FECHA

jOIN SOO POR FECHA, HAY Q CAMBAR LA GRANULARIDAD DE DF.WEATHER A DIA

In [140]:
df_weather.set_index('time', inplace = True)

In [141]:
df_weather = df_weather.resample('D').mean() #TE PONE EL MINIMO DE CADA SEMANA

Hay muxas fechas u no las necesito todas por lo q vamos hacer mas limpieza

In [142]:
#Recoger la parete del data en q  la fecha sea menor o igual a la fecha max q hay en el df
df_weather = df_weather[(df_weather.index <= max(df['FL_DATE'])) & (df_weather.index >= min(df['FL_DATE']))].copy()


In [143]:
df_weather.columns = ['temperature_2m', 'precipitation', 'rain',
       'cloudcover', 'cloudcover_low', 'cloudcover_mid',
       'cloudcover_high', 'windspeed_10m', 'winddirection_10m']
       #renombramos las columnas. Si es exacatamente el nº de columnas q hay lo va a sobreescribir, hay otras formas de hacerlo

In [144]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 730 entries, 2018-01-01 to 2019-12-31
Freq: D
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   temperature_2m     730 non-null    float64
 1   precipitation      730 non-null    float64
 2   rain               730 non-null    float64
 3   cloudcover         730 non-null    float64
 4   cloudcover_low     730 non-null    float64
 5   cloudcover_mid     730 non-null    float64
 6   cloudcover_high    730 non-null    float64
 7   windspeed_10m      730 non-null    float64
 8   winddirection_10m  730 non-null    float64
dtypes: float64(9)
memory usage: 57.0 KB


In [145]:
df_weather['FL_DATE'] = df_weather.index

In [146]:
df = df.join(df_weather, on = 'FL_DATE', how='left', rsuffix='_w')#TODAS LAS COLUMN Q VIENE DE LE METEMOS UN _W

In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250152 entries, 0 to 250151
Data columns (total 25 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   FL_DATE                250152 non-null  datetime64[ns]
 1   OP_CARRIER_AIRLINE_ID  237644 non-null  float64       
 2   TAIL_NUM               235143 non-null  object        
 3   ORIGIN                 250152 non-null  object        
 4   ORIGIN_CITY_NAME       250152 non-null  object        
 5   ORIGIN_STATE_NM        250152 non-null  object        
 6   DEST                   250152 non-null  object        
 7   DEST_CITY_NAME         250152 non-null  object        
 8   DEST_STATE_NM          250152 non-null  object        
 9   CRS_DEP_TIME           250152 non-null  int64         
 10  TAXI_OUT               232641 non-null  float64       
 11  CRS_ARR_TIME           250152 non-null  int64         
 12  ARR_DEL15              250152 non-null  floa

In [148]:
#borramos la columna de fl_date_w
df.drop('FL_DATE_w', axis = 1, inplace = True)

Data Understanding

#### 1. Tamaño del dataset

In [149]:
df.info(verbose=False)
#250152 entries no debe cambiar nunca

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250152 entries, 0 to 250151
Columns: 24 entries, FL_DATE to winddirection_10m
dtypes: datetime64[ns](1), float64(14), int64(2), object(7)
memory usage: 45.8+ MB


# Ver datos

In [150]:
df.head(4).T

,0,1,2,3
FL_DATE,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00,2018-02-01 00:00:00
OP_CARRIER_AIRLINE_ID,"20,363.00","20,363.00","20,363.00","20,363.00"
TAIL_NUM,N299PQ,N920XJ,N605LR,N800AY
ORIGIN,BNA,RDU,BUF,ORF
ORIGIN_CITY_NAME,"Nashville, TN","Raleigh/Durham, NC","Buffalo, NY","Norfolk, VA"
ORIGIN_STATE_NM,Tennessee,North Carolina,New York,Virginia
DEST,JFK,JFK,JFK,JFK
DEST_CITY_NAME,"New York, NY","New York, NY","New York, NY","New York, NY"
DEST_STATE_NM,New York,New York,New York,New York
CRS_DEP_TIME,1359,1354,1529,1704


A continuación se incluye la descripción de las variables:

* **FL_DATE**: Date of the observation
* **OP_CARRIER_AIRLINE_ID**: Identificador de la compañía aérea que opera el vuelo
* **TAIL_NUM**: Número de cola del avión, identificador único del avión
* **ORIGIN**: aeropuerto de origen
* **ORIGIN_CITY_NAME**: ciudad de origen
* **ORIGIN_STATE_NM**: estado (USA) de origen
* **DEST**: aeropuerto de destino, en este caso será siempre el JFK
* **DEST_CITY_NAME**: ciudad de destino, en este caso será siempre Nueva York
* **DEST_STATE_NM**: estado (USA) de destino, en este caso será siempre Nueva York
* **CRS_DEP_TIME**: hora prevista de salida
* **TAXI_OUT**: tiempo transcurrido entre la salida de la terminal y el despegue efectivo del avión
* **CRS_ARR_TIME**: hora prevista de llegada
* **ARR_DEL15**: booleana (1/0) indicando si el vuelo ha llegado con más de 15 minutos de retraso.
* **CRS_ELAPSED_TIME**: tiempo previsto de vuelo.
* **DISTANCE**: distancia entre aeropuertos

Deberemos elegir entre la variable **ARR_DELAY y ARR_DEL15** como predictivos, en este caso, como se trata de un modelo de clasificación, tomaremos como target la variable **ARR_DEL15**


In [151]:
df.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250152 entries, 0 to 250151
Columns: 24 entries, FL_DATE to winddirection_10m
dtypes: datetime64[ns](1), float64(14), int64(2), object(7)
memory usage: 45.8+ MB


In [152]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250152 entries, 0 to 250151
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   FL_DATE                250152 non-null  datetime64[ns]
 1   OP_CARRIER_AIRLINE_ID  237644 non-null  float64       
 2   TAIL_NUM               235143 non-null  object        
 3   ORIGIN                 250152 non-null  object        
 4   ORIGIN_CITY_NAME       250152 non-null  object        
 5   ORIGIN_STATE_NM        250152 non-null  object        
 6   DEST                   250152 non-null  object        
 7   DEST_CITY_NAME         250152 non-null  object        
 8   DEST_STATE_NM          250152 non-null  object        
 9   CRS_DEP_TIME           250152 non-null  int64         
 10  TAXI_OUT               232641 non-null  float64       
 11  CRS_ARR_TIME           250152 non-null  int64         
 12  ARR_DEL15              250152 non-null  floa

In [153]:
df.describe(include=['object']).T

,count,unique,top,freq
TAIL_NUM,235143,2645,N110AN,761
ORIGIN,250152,72,LAX,25345
ORIGIN_CITY_NAME,250152,70,"Los Angeles, CA",25345
ORIGIN_STATE_NM,250152,32,California,51137
DEST,250152,1,JFK,250152
DEST_CITY_NAME,250152,1,"New York, NY",250152
DEST_STATE_NM,250152,1,New York,250152


In [154]:
df.describe(exclude=['object']).T

,count,mean,min,25%,50%,75%,max,std
FL_DATE,250152,2019-01-01 08:55:24.724167424,2018-01-01 00:00:00,2018-07-06 00:00:00,2018-12-30 00:00:00,2019-06-30 00:00:00,2019-12-31 00:00:00,NaN
OP_CARRIER_AIRLINE_ID,"237,644.00","20,152.00","19,690.00","19,790.00","20,363.00","20,409.00","21,171.00",297.88
CRS_DEP_TIME,"250,152.00","1,334.61",3.00,915.00,"1,258.00","1,744.00","2,359.00",529.10
TAXI_OUT,"232,641.00",18.19,1.00,12.00,16.00,21.00,166.00,9.61
CRS_ARR_TIME,"250,152.00","1,419.13",1.00,944.00,"1,455.00","1,914.00","2,400.00",580.99
ARR_DEL15,"250,152.00",0.21,0.00,0.00,0.00,0.00,1.00,0.41
CRS_ELAPSED_TIME,"250,152.00",195.65,47.00,109.00,167.00,306.00,595.00,97.37
DISTANCE,"250,152.00","1,258.84",94.00,427.00,"1,028.00","2,248.00","4,983.00",898.12
temperature_2m,"250,152.00",13.00,-13.85,4.48,13.45,22.01,31.32,9.72
precipitation,"250,152.00",0.15,0.00,0.00,0.00,0.15,2.39,0.29


In [155]:
df.select_dtypes(include=['object']).describe().T

,count,unique,top,freq
TAIL_NUM,235143,2645,N110AN,761
ORIGIN,250152,72,LAX,25345
ORIGIN_CITY_NAME,250152,70,"Los Angeles, CA",25345
ORIGIN_STATE_NM,250152,32,California,51137
DEST,250152,1,JFK,250152
DEST_CITY_NAME,250152,1,"New York, NY",250152
DEST_STATE_NM,250152,1,New York,250152


In [156]:
#nulos, SE debe usar del tipo num no objet
df.select_dtypes(include=['number']).isnull().sum()

,0
OP_CARRIER_AIRLINE_ID,12508
CRS_DEP_TIME,0
TAXI_OUT,17511
CRS_ARR_TIME,0
ARR_DEL15,0
CRS_ELAPSED_TIME,0
DISTANCE,0
temperature_2m,0
precipitation,0
rain,0


**Distribucion del Traget**

Saber los vueles q se retrasan
Si hay null target borrar

In [157]:
df['ARR_DEL15'].mean() #TARGET

0.21040007675333397

In [158]:
# 21.04 vuelos q se retrasan
#vigilad si el % es muy bajo o alto porq se complica la modelización

In [161]:
df['ARR_DEL15'].value_counts()

,count
ARR_DEL15,
0.00,197520
1.00,52632


EDAD

In [164]:
#Fechas
LAX_df_dia = LAX_df[['FL_DATE','ARR_DEL15']].groupby(['FL_DATE']).mean()

In [166]:
#Si hay dias donde no tengo casi retrasos otros dias muxos
LAX_df_dia.head()

,ARR_DEL15
FL_DATE,
2018-01-01,0.32
2018-01-02,0.37
2018-01-03,0.11
2018-01-04,0.50
2018-01-05,0.72
